In [76]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.types import IntegerType, LongType, FloatType, DoubleType
from pyspark.sql.functions import col
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.linalg import Vectors


spark = SparkSession.builder \
    .appName("StrokeDataAnalysis") \
        .getOrCreate()

df = pd.read_csv("./stroke_data.csv")
df_spark = spark.read.csv('./stroke_data.csv')

print(df.head())
df_spark.printSchema()


24/06/13 16:02:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


   0  gender   age  hypertension  heart_disease ever_married      work_type  \
0  1  Female  18.0             0              0           No        Private   
1  2    Male  58.0             1              0          Yes        Private   
2  3  Female  36.0             0              0          Yes       Govt_job   
3  4  Female  62.0             0              0          Yes  Self-employed   
4  5  Female  82.0             0              0          Yes        Private   

  Residence_type  avg_glucose_level    bmi   smoking_status  stroke  
0          Urban              94.19  12.12           smokes       1  
1          Rural             154.24  33.70     never_smoked       0  
2          Urban              72.63  24.70           smokes       0  
3          Rural              85.52  31.20  formerly smoked       0  
4          Rural              59.32  33.20           smokes       1  
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = t

In [77]:
spark = SparkSession.builder \
    .appName("Proporção de participantes por gênero") \
    .getOrCreate()

df = spark.read.csv("./stroke_data.csv", header=True, inferSchema=True)

df = pd.read_csv("./stroke_data.csv")

print(df.head())


24/06/13 16:02:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


   0  gender   age  hypertension  heart_disease ever_married      work_type  \
0  1  Female  18.0             0              0           No        Private   
1  2    Male  58.0             1              0          Yes        Private   
2  3  Female  36.0             0              0          Yes       Govt_job   
3  4  Female  62.0             0              0          Yes  Self-employed   
4  5  Female  82.0             0              0          Yes        Private   

  Residence_type  avg_glucose_level    bmi   smoking_status  stroke  
0          Urban              94.19  12.12           smokes       1  
1          Rural             154.24  33.70     never_smoked       0  
2          Urban              72.63  24.70           smokes       0  
3          Rural              85.52  31.20  formerly smoked       0  
4          Rural              59.32  33.20           smokes       1  


In [78]:
#QUESTAO 1:

num_registros = df.shape[0]
print(f'Número de registro: {num_registros}')

Número de registro: 67135


In [79]:
#QUESTAO 2:

total_colunas = len(df.columns)
print(f'Número total de colunas: {total_colunas}')

spark = SparkSession.builder \
    .appName("Stroke Data Analysis") \
    .getOrCreate()
    
df = spark.read.csv('./stroke_data.csv', header=True, inferSchema=True)


df.printSchema()

numeric_types = (IntegerType, LongType, FloatType, DoubleType)
num_numeric_columns = len([field for field in df.schema.fields if isinstance(field.dataType, numeric_types)])


print(f'Número de colunas numéricas: {num_numeric_columns}')


Número total de colunas: 12


24/06/13 16:02:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)

Número de colunas numéricas: 7


In [80]:
# QUESTAO 3:

stroke_counts = df.groupBy("stroke").count()
stroke_counts.show()


+------+-----+
|stroke|count|
+------+-----+
|     1|40287|
|     0|26848|
+------+-----+



In [81]:
# QUESTÃO 4:

df.createOrReplaceTempView('stroke_table')

query = """
SELECT work_type, COUNT(*) as stroke_count
FROM stroke_table
WHERE stroke = 1
GROUP BY work_type
"""

result = spark.sql(query)
result.show()


+-------------+------------+
|    work_type|stroke_count|
+-------------+------------+
| Never_worked|          85|
|Self-employed|       10807|
|      Private|       23711|
|     children|         520|
|     Govt_job|        5164|
+-------------+------------+



In [82]:
#QUESTÃO 5:


df.createOrReplaceTempView("stroke_data")
consulta = """
SELECT 
    gender,
    COUNT(*) AS total_participantes,
    COUNT(*) / (SELECT COUNT(*) FROM stroke_data) AS proporcao
FROM 
    stroke_data
GROUP BY 
    gender
"""

resultado = spark.sql(consulta)

resultado.show()

+------+-------------------+--------------------+
|gender|total_participantes|           proporcao|
+------+-------------------+--------------------+
|Female|              39530|  0.5888135845684069|
| Other|                 11|1.638489610486333...|
|  Male|              27594|  0.4110225664705444|
+------+-------------------+--------------------+



In [83]:
# QUESTÃO 6:

df.createOrReplaceTempView("stroke_data")

consulta_hipertensos = """
SELECT 
    COUNT(*) AS total_hipertensos_com_derrame,
    COUNT(*) / (SELECT COUNT(*) FROM stroke_data WHERE hypertension = 1) AS percentual_hipertensos_com_derrame
FROM 
    stroke_data
WHERE 
    hypertension = 1
    AND stroke = 1
"""

resultado_hipertensos = spark.sql(consulta_hipertensos)

resultado_hipertensos.show()

consulta_nao_hipertensos = """
SELECT 
    COUNT(*) AS total_nao_hipertensos_com_derrame,
    COUNT(*) / (SELECT COUNT(*) FROM stroke_data WHERE hypertension = 0) AS percentual_nao_hipertensos_com_derrame
FROM 
    stroke_data
WHERE 
    hypertension = 0
    AND stroke = 1
"""

resultado_nao_hipertensos = spark.sql(consulta_nao_hipertensos)

resultado_nao_hipertensos.show()

+-----------------------------+----------------------------------+
|total_hipertensos_com_derrame|percentual_hipertensos_com_derrame|
+-----------------------------+----------------------------------+
|                         8817|                0.8003086139602432|
+-----------------------------+----------------------------------+

+---------------------------------+--------------------------------------+
|total_nao_hipertensos_com_derrame|percentual_nao_hipertensos_com_derrame|
+---------------------------------+--------------------------------------+
|                            31470|                    0.5607826365871913|
+---------------------------------+--------------------------------------+



In [84]:
# QUESTAO 7:

consulta = """
SELECT 
    age,
    COUNT(*) AS total_pessoas_com_derrame
FROM 
    stroke_data
WHERE 
    stroke = 1
GROUP BY 
    age
ORDER BY 
    total_pessoas_com_derrame DESC
"""

resultado = spark.sql(consulta)

resultado.show()

idade_maior_numero_derrames = resultado.select("age").first()[0]
print(f"A idade com o maior número de pessoas que sofreram derrame é: {idade_maior_numero_derrames}")

+----+-------------------------+
| age|total_pessoas_com_derrame|
+----+-------------------------+
|79.0|                     2916|
|78.0|                     2279|
|80.0|                     1858|
|81.0|                     1738|
|82.0|                     1427|
|77.0|                      994|
|74.0|                      987|
|63.0|                      942|
|76.0|                      892|
|70.0|                      881|
|66.0|                      848|
|75.0|                      809|
|67.0|                      801|
|57.0|                      775|
|73.0|                      759|
|65.0|                      716|
|72.0|                      709|
|68.0|                      688|
|69.0|                      677|
|71.0|                      667|
+----+-------------------------+
only showing top 20 rows

A idade com o maior número de pessoas que sofreram derrame é: 79.0


In [85]:
# QUESTAO 8:
pessoas_com_derrame_apos_50 = df.filter((col("age") > 50) & (col("stroke") == 1))

total_pessoas_com_derrame_apos_50 = pessoas_com_derrame_apos_50.count()

print(f"Total de pessoas que sofreram derrame após os 50 anos: {total_pessoas_com_derrame_apos_50}")

Total de pessoas que sofreram derrame após os 50 anos: 28938


In [86]:
# QUESTAO 9:

consulta_com_derrame = """
SELECT 
    AVG(avg_glucose_level) AS nivel_medio_glicose_com_derrame
FROM 
    stroke_data
WHERE 
    stroke = 1
"""

resultado_com_derrame = spark.sql(consulta_com_derrame)

nivel_medio_glicose_com_derrame = resultado_com_derrame.first()[0]
print(f"Nível médio de glicose para pessoas que sofreram derrame: {nivel_medio_glicose_com_derrame:.2f}")

consulta_sem_derrame = """
SELECT 
    AVG(avg_glucose_level) AS nivel_medio_glicose_sem_derrame
FROM 
    stroke_data
WHERE 
    stroke = 0
"""

resultado_sem_derrame = spark.sql(consulta_sem_derrame)

nivel_medio_glicose_sem_derrame = resultado_sem_derrame.first()[0]
print(f"Nível médio de glicose para pessoas que não sofreram derrame: {nivel_medio_glicose_sem_derrame:.2f}")


Nível médio de glicose para pessoas que sofreram derrame: 119.95
Nível médio de glicose para pessoas que não sofreram derrame: 103.60


In [87]:
# QUESTAO 10:

consulta_com_derrame = """
SELECT 
    AVG(bmi) AS bmi_medio_com_derrame
FROM 
    stroke_data
WHERE 
    stroke = 1
"""

resultado_com_derrame = spark.sql(consulta_com_derrame)

bmi_medio_com_derrame = resultado_com_derrame.first()[0]
print(f"BMI médio para pessoas que sofreram derrame: {bmi_medio_com_derrame:.2f}")

consulta_sem_derrame = """
SELECT 
    AVG(bmi) AS bmi_medio_sem_derrame
FROM 
    stroke_data
WHERE 
    stroke = 0
"""

resultado_sem_derrame = spark.sql(consulta_sem_derrame)

bmi_medio_sem_derrame = resultado_sem_derrame.first()[0]
print(f"BMI médio para pessoas que não sofreram derrame: {bmi_medio_sem_derrame:.2f}")

BMI médio para pessoas que sofreram derrame: 29.94
BMI médio para pessoas que não sofreram derrame: 27.99


In [88]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [89]:
# QUESTÃO 11:

spark = SparkSession.builder \
    .appName("Modelo de Árvore de Decisão para Previsão de Stroke") \
    .getOrCreate()

df = spark.read.csv('./stroke_data.csv', header=True, inferSchema=True)

data = df.select("age", "bmi", "hypertension", "heart_disease", "avg_glucose_level", "stroke")

feature_columns = ["age", "bmi", "hypertension", "heart_disease", "avg_glucose_level"]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)

train_data, test_data = data.randomSplit([0.8, 0.2], seed=1234)

dt = DecisionTreeClassifier(labelCol="stroke", featuresCol="features", seed=1234)
model = dt.fit(train_data)

predictions = model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="stroke")
accuracy = evaluator.evaluate(predictions)

print(f"Acurácia do modelo: {accuracy:.4f}")



24/06/13 16:02:42 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Acurácia do modelo: 0.6483


In [90]:
# questao 12:

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline

spark = SparkSession.builder \
    .appName("Modelo de Árvore de Decisão para Previsão de Stroke") \
    .getOrCreate()

df = spark.read.csv('stroke_data.csv', header=True, inferSchema=True)

data = df.select("age", "bmi", "hypertension", "heart_disease", "avg_glucose_level", "gender", "smoking_status", "stroke")

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="skip") for column in ["gender", "smoking_status"]]
encoder = [OneHotEncoder(inputCol=column+"_index", outputCol=column+"_encoded") for column in ["gender", "smoking_status"]]
assembler = VectorAssembler(inputCols=["age", "bmi", "hypertension", "heart_disease", "avg_glucose_level", "gender_encoded", "smoking_status_encoded"], outputCol="features")

pipeline = Pipeline(stages=indexers + encoder + [assembler])

pipeline_model = pipeline.fit(data)
transformed_data = pipeline_model.transform(data)

train_data, test_data = transformed_data.randomSplit([0.8, 0.2], seed=1234)

dt = DecisionTreeClassifier(labelCol="stroke", featuresCol="features", seed=1234)

model = dt.fit(train_data)

predictions = model.transform(test_data)

from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="stroke")
accuracy = evaluator.evaluate(predictions)

print(f"Acurácia do modelo: {accuracy:.4f}")



Acurácia do modelo: 0.7947


In [91]:
# QUESTÃO 13:

importances = {feature: feature_importances[i] for i, feature in enumerate(feature_columns)}
most_important_feature = max(importances, key=importances.get)
print(f"A variável mais importante no modelo é: {most_important_feature}")

A variável mais importante no modelo é: age


In [92]:
# questao 14:

tree_depth = model.depth
print(f"A profundidade da árvore de decisão é: {tree_depth}")

A profundidade da árvore de decisão é: 5


In [93]:
# QUESTAO 15:


num_nodes = model.numNodes
print(f"O número de nodos na árvore de decisão é: {num_nodes}")


O número de nodos na árvore de decisão é: 19
